In [1]:
import numpy as np

In [2]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [3]:
a = [0.1, 0.2, 0.5, 0.3]
reweight_distribution(a, 1.0)

array([0.09090909, 0.18181818, 0.45454545, 0.27272727])

In [4]:
reweight_distribution(a, 0.01)

array([1.26765060e-70, 1.60693804e-40, 1.00000000e+00, 6.53318624e-23])

In [5]:
reweight_distribution(a, 0.4)

array([0.01279639, 0.07238734, 0.71534021, 0.19947605])

In [6]:
reweight_distribution(a, 10000.0)

array([0.24997874, 0.24999607, 0.25001898, 0.25000621])

In [7]:
import keras
import numpy as np

Using TensorFlow backend.


In [8]:
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 1s 2us/step
Corpus length: 600893


In [9]:
maxlen = 60
step = 3
sentences = []
next_chars = []

In [10]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [11]:
len(sentences)

200278

In [12]:
chars = sorted(list(set(text)))
len(chars)

57

In [15]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [17]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [18]:
from keras import layers

In [19]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [21]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [25]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch:', epoch)
    
    model.fit(x, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Generating with seed:' + generated_text)
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.0
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()

## DeepDream

In [1]:
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0)

In [3]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 38s 0us/step


In [ ]:
model.summary()

In [5]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.0,
    'mixed4': 2.0,
    'mixed5': 1.5
}

In [6]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [9]:
layer_dict['mixed2'].output

<tf.Tensor 'mixed2/concat:0' shape=(None, None, None, 288) dtype=float32>

In [29]:
loss = K.variable(0.0)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff + K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling

In [30]:
loss

<tf.Tensor 'add_20:0' shape=() dtype=float32>

In [31]:
dream = model.input

In [32]:
dream

<tf.Tensor 'input_1:0' shape=(None, None, None, 3) dtype=float32>

In [33]:
grads = K.gradients(loss, dream)[0]

In [34]:
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

In [35]:
outputs = [loss, grads]

In [36]:
fetch_loss_and_grads = K.function([dream], outputs)

In [38]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grads_values = outs[1]
    return loss_value, grads_values

In [62]:
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('I’m ...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [44]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.0
base_image_path = './05.jpg'

In [46]:
import scipy
from keras.preprocessing import image


def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

img = preprocess_image(base_image_path)

In [49]:
img.shape

(1, 420, 560, 3)

In [51]:
original_shape = img.shape[1:3]
original_shape

(420, 560)

In [55]:
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

In [58]:
def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [60]:
shrunk_original_img.shape

(1, 214, 285, 3)

In [64]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.0
    x += 0.5
    x *= 255.0
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [74]:
from imageio import imsave
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imsave(fname, pil_img)

In [75]:
for shape in successive_shapes:
    print('I’m Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

I’m Processing image shape (214, 285)
I’m Processing image shape (300, 400)
I’m ...Loss value at 0 : 8.0134945
I’m ...Loss value at 1 : 8.460516
I’m ...Loss value at 2 : 8.860331
I’m ...Loss value at 3 : 9.258212
I’m ...Loss value at 4 : 9.6242895
I’m ...Loss value at 5 : 9.983233
I’m Processing image shape (420, 560)
I’m ...Loss value at 0 : 8.102766
I’m ...Loss value at 1 : 8.593761
I’m ...Loss value at 2 : 9.033359
I’m ...Loss value at 3 : 9.445826
I’m ...Loss value at 4 : 9.837204
